## Config

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *

## GDAX

In [2]:
# https://github.com/danpaquin/gdax-python
# https://docs.gdax.com

# Use the sandbox API (requires a different set of API access credentials)
gdax_client = gdax.AuthenticatedClient(cfg.GDAX_API_KEY, cfg.GDAX_API_SECRET_KEY, 
                                       cfg.GDAX_PASSPHRASE, api_url=cfg.GDAX_ENDPOINT)
gdax_client.get_time()

{'epoch': 1514664510.852, 'iso': '2017-12-30T20:08:30.852Z'}

### Historical Prices

In [11]:
prices = gdax_client.get_product_historic_rates(c.BTC_USD)
prices

[[1514665260, 13239, 13240, 13240, 13239, 0.9701913099999999],
 [1514665200, 13240, 13266.39, 13250, 13240, 7.2483495],
 [1514665140, 13250, 13290.01, 13290.01, 13250, 11.455645239999999],
 [1514665080, 13290, 13290.01, 13290, 13290.01, 5.384689460000001],
 [1514665020, 13290, 13341.98, 13341.97, 13290.01, 12.136442039999999],
 [1514664960, 13300, 13341.98, 13341, 13341.97, 7.83809546],
 [1514664900, 13341, 13341.16, 13341.16, 13341, 3.4663720799999997],
 [1514664840, 13341.15, 13341.16, 13341.15, 13341.16, 3.7167261],
 [1514664780, 13341.15, 13341.16, 13341.16, 13341.16, 4.1298974],
 [1514664720, 13341.15, 13341.16, 13341.16, 13341.16, 4.14397738],
 [1514664660, 13341.14, 13341.61, 13341.14, 13341.15, 3.2233896599999996],
 [1514664600, 13322.97, 13370.82, 13322.98, 13340.01, 10.038340099999997],
 [1514664540, 13300.01, 13330.83, 13330.83, 13322.98, 14.79310236],
 [1514664480, 13320, 13350, 13350, 13330.83, 2.07595344],
 [1514664420, 13349.99, 13350, 13349.99, 13350, 3.2762585700000004

In [ ]:
# December 1, 2015
START_UTC_ISO = datetime.datetime.strptime(
    '2015-12-01T00:00:00Z', '%Y-%m-%dT%H:%M:%SZ').astimezone(datetime.timezone.utc)
END_UTC_ISO = datetime.datetime.now(tz=datetime.timezone.utc)
BTC_PRICE_FPATH = os.path.join(cfg.DATA_DIR, 'gdax_btc_price.csv')
PRICE_COLUMNS = ['time', 'low', 'high', 'open', 'close', 'volume']

def write_to_df(data, fpath):
    if os.path.exists(fpath):
        df = pd.read_csv(fpath)
        data = pd.DataFrame(data, columns=PRICE_COLUMNS)
        df.set_index('time')
        df = df.sort_values(by='time', ascending=True)
        df = pd.concat([df, data]).drop_duplicates().reset_index(drop=True)
    else:
        df = pd.DataFrame(data, columns=PRICE_COLUMNS)
        df.set_index('time')
        df = df.sort_values(by='time', ascending=True)
    df.to_csv(fpath, index=False)
    return df

def get_data(currency_pair, start_utc, end_utc, timestep_sec):
    # how to handle missing values?
#     print("Start", start_utc.isoformat())
#     print("End", end_utc.isoformat())
    delta = end_utc - start_utc
#     print("Years", round(delta.days/365,2))
#     print("Days", delta.days)
#     print("Hours", delta.days*24)
#     print("Minutes", delta.days*24*60)
#     print("Seconds", round(delta.total_seconds()))
    data = gdax_client.get_product_historic_rates(
        currency_pair, start=start_utc.isoformat(), 
        end=end_utc.isoformat(), granularity=timestep_sec)
    return data

def get_all_data(pair, start_utc, end_utc, timesteps_per_request, timestep_sec, outfpath):
    cur_utc = start_utc
    print("Start", cur_utc)
    print("End", end_utc)
    time_delta = datetime.timedelta(
        seconds=timesteps_per_request*timestep_sec)
    n_records = 0
    while cur_utc < end_utc:
        data = get_data(pair, cur_utc, cur_utc+time_delta, timestep_sec)
        time.sleep(1)
        cur_utc = datetime.datetime.fromtimestamp(
            data[0][0] + timestep_sec).astimezone(datetime.timezone.utc)
        df = write_to_df(data, outfpath)
        n_records += len(data)
        print("Records", n_records)
    return df

df = get_all_data(c.BTC_USD, START_UTC_ISO, END_UTC_ISO, 400, 60, BTC_PRICE_FPATH)

Start 2015-12-01 08:00:00+00:00
End 2017-12-30 22:11:13.719504+00:00
Records 386
Records 783
Records 1178
Records 1552
Records 1939
Records 2331
Records 2716
Records 3092
Records 3490
Records 3887
Records 4265
Records 4632
Records 5029
Records 5427
Records 5774
Records 6163
Records 6558
Records 6951
Records 7307
Records 7703
Records 8100
Records 8463
Records 8852
Records 9245
Records 9614
Records 9976
Records 10370
Records 10768
Records 11158
Records 11545
Records 11944
Records 12340
Records 12694
Records 13085
Records 13484
Records 13876
Records 14269
Records 14668
Records 15068
Records 15463
Records 15848
Records 16241
Records 16624
Records 16922
Records 17301
Records 17698
Records 18069
Records 18464
Records 18779
Records 19178
Records 19568
Records 19967
Records 20367
Records 20763
Records 21163
Records 21560
Records 21960
Records 22344
Records 22727
Records 23039
Records 23215
Records 23395
Records 23581
Records 23976
Records 24344
Records 24688
Records 25086
Records 25473
Records

In [107]:
df.join()

,time,low,high,open,close,volume
0,1448980740,360.48,360.49,360.48,360.49,15.132200
1,1448980680,360.48,360.49,360.48,360.49,4.106300
2,1448980620,360.49,360.49,360.49,360.49,10.085500
3,1448980560,360.48,360.48,360.48,360.48,0.692549
4,1448980500,360.48,360.49,360.48,360.49,3.094100
5,1448980440,360.49,360.49,360.49,360.49,22.535286
6,1448980380,360.48,360.49,360.48,360.49,17.980500
7,1448980320,360.48,360.49,360.48,360.49,18.200400
8,1448980260,360.49,360.49,360.49,360.49,3.261100
9,1448980200,360.49,360.49,360.49,360.49,2.241400


In [98]:
all_data[0],len(all_data)

([1448980740, 360.48, 360.49, 360.48, 360.49, 15.132200000000001], 1939)

In [ ]:
delta.total_seconds(), delta.seconds

In [30]:
end = datetime.datetime.now(tz=datetime.timezone.utc).isoformat()
start = (now - datetime.timedelta(seconds=3600*2)).isoformat()
print(start, end)

2017-12-30T18:11:21.044835+00:00 2017-12-30T20:37:54.232586+00:00


In [25]:
# Default request is 400 minutes (~7 hours), 1 minute gap
# ~4 requests to get 24 hours of data
# 1460 requests for 1 year, ~3000 requests for 2 years

s = datetime.datetime.fromtimestamp(1514641260)
next_ = datetime.datetime.fromtimestamp(1514641320)
e = datetime.datetime.fromtimestamp(1514665260)
s,next_,e,len(prices),(e-s).total_seconds()/3600

(datetime.datetime(2017, 12, 30, 5, 41),
 datetime.datetime(2017, 12, 30, 5, 42),
 datetime.datetime(2017, 12, 30, 12, 21),
 401,
 6.666666666666667)

In [74]:
datetime.datetime.timestamp(s)

1514641260.0

In [12]:
len(prices)

401

In [9]:
end = datetime.datetime.now(tz=datetime.timezone.utc)
start = now - datetime.timedelta(seconds=3600*2)
print(start, end)
gdax_client.get_product_historic_rates(
    c.BTC_USD, start=start.isoformat(), 
    end=end.isoformat(), granularity=3600)

2017-12-30 18:11:21.044835+00:00 2017-12-30 20:14:51.402593+00:00


[[1514664000, 13407.66, 13475.69, 13464.73, 13450, 28.54695655999999],
 [1514660400, 13083.98, 13490, 13083.99, 13465.01, 361.428495399999],
 [1514656800, 12850.02, 13295.99, 13284, 13083.99, 468.2405779299974],
 [1514653200, 12550, 13433.97, 12575, 13283.99, 533.1549132499985],
 [1514649600, 12500, 13150, 13049.26, 12575, 502.9561247299981],
 [1514646000, 12900.01, 13332, 13320.24, 13067, 478.2737538199964],
 [1514642400, 12941.38, 13596.68, 13294.95, 13320.23, 314.9257114000012],
 [1514638800, 12845, 13500.01, 13500.01, 13262.73, 469.3368624899971],
 [1514635200, 13500, 13893.89, 13888.57, 13500, 221.9694639799984],
 [1514631600, 13740.01, 13968.58, 13768.01, 13888.57, 153.61877297999868],
 [1514628000, 13750.1, 13950, 13912, 13768, 175.2285511199987],
 [1514624400, 13837.4, 13985.58, 13936.21, 13912, 159.25136961999874],
 [1514620800, 13929.5, 14100, 14069.8, 13929.5, 194.1644011699984],
 [1514617200, 13905, 14069.81, 13950.02, 14069.81, 164.92757911999897],
 [1514613600, 13946.01, 

### T

### Exchange Metadata

In [ ]:
gdax_client.get_products()
gdax_client.get_currencies()
gdax_client.get_time()

### Current Prices

In [ ]:
# Get the order book at the default level.
gdax_client.get_product_order_book('BTC-USD')
# Get the order book at a specific level.
gdax_client.get_product_order_book('BTC-USD', level=1)

In [ ]:
# Get the product ticker for a specific product.
gdax_client.get_product_ticker(product_id='ETH-USD')

In [ ]:
# Get the product trades for a specific product.
gdax_client.get_product_trades(product_id='ETH-USD')

In [ ]:
gdax_client.get_product_24hr_stats('ETH-USD')

### Streaming

In [ ]:
class myWebsocketClient(gdax.WebsocketClient):
    def on_open(self):
        self.url = cfg.GDAX_WEBSOCKET
        self.products = [c.BTC_USD]
        self.message_count = 0
        print("Lets count the messages!")
    
    def on_message(self, msg):
        self.message_count += 1
        if 'price' in msg and 'type' in msg:
            print ("Message type:", msg["type"],
                   "\t@ {:.3f}".format(float(msg["price"])))
    
    def on_close(self):
        print("-- Goodbye! --")

        
wsClient = myWebsocketClient()
wsClient.start()
print(wsClient.url, wsClient.products)
while (wsClient.message_count < 500):
    print ("\nmessage_count =", "{} \n".format(wsClient.message_count))
    time.sleep(1)
wsClient.close()

In [ ]:
wsClient.close()